# 기본설정

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import konlpy
from konlpy.tag import Mecab
from konlpy.tag import Okt

import fasttext
from gensim.models import Word2Vec, FastText

In [2]:
bills = pd.read_csv('bills_24839_all_cleansed.csv')

In [3]:
members = pd.read_csv('assembly_memebers_295.csv')

In [4]:
bills.shape

(23779, 7)

In [5]:
bills.head()

,입안일,법안명,법안내용,발의자,상태,법안명_이름만,법안명_법안내용
0,2020-02-24,출입국관리법 일부개정법률안(정성호의원 등 10인),관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,정성호(더불어민주당/鄭成湖) 김민기(더불어민주당/金敏基) 김병관(더불어민주당/金炳官...,계류,출입국관리법 일부개정법률안,출입국관리법 일부개정법률안 관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리...
1,2020-02-24,통신비밀보호법 일부개정법률안(추혜선의원 등 10인),2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,추혜선(정의당/秋惠仙) 김종대(정의당/金鍾大) 김종훈(민중당/金鍾勳) 박선숙(바른미...,계류,통신비밀보호법 일부개정법률안,통신비밀보호법 일부개정법률안 2018. 8. 30. 헌법재판소는 국가정보원 인터넷...
2,2020-02-24,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안(신창현의원 등 ...,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...",신창현(더불어민주당/申昌賢) 김경협(더불어민주당/金炅俠) 김병기(더불어민주당/金炳基...,계류,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안 현행 법령상 ...
3,2020-02-21,군인사법 일부개정법률안(정부),일과 가정의 양립이 가능한 공직문화 조성을 위하여 첫째 자녀에 대하여 부모가 모두 ...,정부,계류,군인사법 일부개정법률안,군인사법 일부개정법률안 일과 가정의 양립이 가능한 공직문화 조성을 위하여 첫째 자녀...
4,2020-02-20,중소기업진흥에 관한 법률 일부개정법률안(서영교의원 등 10인),최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,서영교(더불어민주당/徐瑛敎) 김상희(더불어민주당/金相姬) 김철민(더불어민주당/金哲玟...,계류,중소기업진흥에 관한 법률 일부개정법률안,중소기업진흥에 관한 법률 일부개정법률안 최근 코로나바이러스감염증-19(COVID-...


## 다듬기

In [6]:
bills = bills[(bills['발의자']!='위원장') & (bills['발의자'] != '정부')]

In [7]:
bills.drop(columns='법안명', inplace=True)

In [8]:
bills.rename(columns={"법안명_이름만":"법안명", "법안명_법안내용":"법안명,법안내용"},inplace=True)

In [9]:
bills.columns

Index(['입안일', '법안내용', '발의자', '상태', '법안명', '법안명,법안내용'], dtype='object')

In [10]:
bills = bills[['입안일', '법안명', '법안내용', '발의자', '상태']]

In [11]:
bills.reset_index(inplace=True)

In [12]:
bills.head()

,index,입안일,법안명,법안내용,발의자,상태
0,0,2020-02-24,출입국관리법 일부개정법률안,관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,정성호(더불어민주당/鄭成湖) 김민기(더불어민주당/金敏基) 김병관(더불어민주당/金炳官...,계류
1,1,2020-02-24,통신비밀보호법 일부개정법률안,2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,추혜선(정의당/秋惠仙) 김종대(정의당/金鍾大) 김종훈(민중당/金鍾勳) 박선숙(바른미...,계류
2,2,2020-02-24,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...",신창현(더불어민주당/申昌賢) 김경협(더불어민주당/金炅俠) 김병기(더불어민주당/金炳基...,계류
3,4,2020-02-20,중소기업진흥에 관한 법률 일부개정법률안,최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,서영교(더불어민주당/徐瑛敎) 김상희(더불어민주당/金相姬) 김철민(더불어민주당/金哲玟...,계류
4,5,2020-02-19,주거급여법 일부개정법률안,현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...,함진규(자유한국당/咸珍圭) 강석진(자유한국당/姜錫振) 김성원(자유한국당/金成願) 김...,계류


In [13]:
bills.reset_index(inplace=True)
bills.head()

,level_0,index,입안일,법안명,법안내용,발의자,상태
0,0,0,2020-02-24,출입국관리법 일부개정법률안,관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,정성호(더불어민주당/鄭成湖) 김민기(더불어민주당/金敏基) 김병관(더불어민주당/金炳官...,계류
1,1,1,2020-02-24,통신비밀보호법 일부개정법률안,2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,추혜선(정의당/秋惠仙) 김종대(정의당/金鍾大) 김종훈(민중당/金鍾勳) 박선숙(바른미...,계류
2,2,2,2020-02-24,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...",신창현(더불어민주당/申昌賢) 김경협(더불어민주당/金炅俠) 김병기(더불어민주당/金炳基...,계류
3,3,4,2020-02-20,중소기업진흥에 관한 법률 일부개정법률안,최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,서영교(더불어민주당/徐瑛敎) 김상희(더불어민주당/金相姬) 김철민(더불어민주당/金哲玟...,계류
4,4,5,2020-02-19,주거급여법 일부개정법률안,현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...,함진규(자유한국당/咸珍圭) 강석진(자유한국당/姜錫振) 김성원(자유한국당/金成願) 김...,계류


In [14]:
bills.dropna(axis=0, inplace=True)

In [ ]:
bills.drop(columns=['index', 'level_0'], inplace=True)
bills.reset_index(inplace=True)

In [17]:
bills.drop(columns='index', inplace=True)
bills.head()

,입안일,법안명,법안내용,발의자,상태
0,2020-02-24,출입국관리법 일부개정법률안,관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,정성호(더불어민주당/鄭成湖) 김민기(더불어민주당/金敏基) 김병관(더불어민주당/金炳官...,계류
1,2020-02-24,통신비밀보호법 일부개정법률안,2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,추혜선(정의당/秋惠仙) 김종대(정의당/金鍾大) 김종훈(민중당/金鍾勳) 박선숙(바른미...,계류
2,2020-02-24,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...",신창현(더불어민주당/申昌賢) 김경협(더불어민주당/金炅俠) 김병기(더불어민주당/金炳基...,계류
3,2020-02-20,중소기업진흥에 관한 법률 일부개정법률안,최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,서영교(더불어민주당/徐瑛敎) 김상희(더불어민주당/金相姬) 김철민(더불어민주당/金哲玟...,계류
4,2020-02-19,주거급여법 일부개정법률안,현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...,함진규(자유한국당/咸珍圭) 강석진(자유한국당/姜錫振) 김성원(자유한국당/金成願) 김...,계류


In [21]:
bills.reset_index(inplace=True)
bills.head()

,level_0,index,입안일,법안명,법안내용,발의자,상태
0,0,0,2020-02-24,출입국관리법 일부개정법률안,관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,정성호(더불어민주당/鄭成湖) 김민기(더불어민주당/金敏基) 김병관(더불어민주당/金炳官...,계류
1,1,1,2020-02-24,통신비밀보호법 일부개정법률안,2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,추혜선(정의당/秋惠仙) 김종대(정의당/金鍾大) 김종훈(민중당/金鍾勳) 박선숙(바른미...,계류
2,2,2,2020-02-24,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...",신창현(더불어민주당/申昌賢) 김경협(더불어민주당/金炅俠) 김병기(더불어민주당/金炳基...,계류
3,3,3,2020-02-20,중소기업진흥에 관한 법률 일부개정법률안,최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,서영교(더불어민주당/徐瑛敎) 김상희(더불어민주당/金相姬) 김철민(더불어민주당/金哲玟...,계류
4,4,4,2020-02-19,주거급여법 일부개정법률안,현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...,함진규(자유한국당/咸珍圭) 강석진(자유한국당/姜錫振) 김성원(자유한국당/金成願) 김...,계류


In [22]:
bills.to_csv("bills_21449_index.csv", index=False)

## 완료!!!

# 나누기 시작

In [23]:
df = pd.read_csv("bills_21449_index.csv")

In [24]:
hw = bills[['index','법안명', '법안내용']]

In [26]:
hw.head()

,index,법안명,법안내용
0,0,출입국관리법 일부개정법률안,관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...
1,1,통신비밀보호법 일부개정법률안,2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...
2,2,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해..."
3,3,중소기업진흥에 관한 법률 일부개정법률안,최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...
4,4,주거급여법 일부개정법률안,현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...


## tokenized 진행

In [28]:
mecab = Mecab()
def tokenized_mecab(data):
    result = mecab.morphs(data)
    result = ' '.join(result)
    return result

In [30]:
hw.head()

,index,법안명,법안내용
0,0,출입국관리법 일부개정법률안,관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...
1,1,통신비밀보호법 일부개정법률안,2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...
2,2,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해..."
3,3,중소기업진흥에 관한 법률 일부개정법률안,최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...
4,4,주거급여법 일부개정법률안,현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...


In [32]:
hw['tokenized'] = hw['법안내용'].apply(tokenized_mecab)

<ipython-input-32-ba3c894a690e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hw['tokenized'] = hw['법안내용'].apply(tokenized_mecab)


### tokenized 완료

In [33]:
hw.head()

,index,법안명,법안내용,tokenized
0,0,출입국관리법 일부개정법률안,관광 등 단기체류 목적으로 입국하는 외국인은 출입국관리공무원의 입국심사를 받을 때...,관광 등 단기체류 목적 으로 입국 하 는 외국인 은 출입국관리공무원 의 입국 심사 ...
1,1,통신비밀보호법 일부개정법률안,2018. 8. 30. 헌법재판소는 국가정보원 인터넷회선감청(이른바 ‘패킷감청’)...,2018 . 8 . 30 . 헌법재판소 는 국가정보원 인터넷 회선 감청 ( 이른바 ...
2,2,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,"현행 법령상 도시가스 미공급 지역의 경우, 도시가스 사업자가 도시가스 공급을 위해...","현행 법령 상 도시 가스 미공 급 지역 의 경우 , 도시 가스 사업자 가 도시 가스..."
3,3,중소기업진흥에 관한 법률 일부개정법률안,최근 코로나바이러스감염증-19(COVID-19)의 확산으로 직·간접적으로 경제적 ...,최근 코로나 바이러스 감염증 - 19 ( COVID - 19 ) 의 확산 으로 직 ...
4,4,주거급여법 일부개정법률안,현행법은 「국민기초생활 보장법」 관련 규정을 준용하여 주거급여를 실시하기 위한 비...,현행법 은 「 국민 기초 생활 보장 법 」 관련 규정 을 준용 하 여 주거급여 를 ...


In [34]:
hw.shape

(21449, 4)

### 각자 30개의 법안을 맡아라!!!

In [35]:
randint = np.random.randint(1, 21449, size=120)
토끼num = randint[:30]
참새num = randint[30:60]
병아리num = randint[60:90]
거북이num = randint[90:120]

In [36]:
df_토끼 = hw.loc[토끼num]
df_참새 = hw.loc[참새num]
df_병아리 = hw.loc[병아리num]
df_거북이 = hw.loc[거북이num]

In [37]:
df_토끼.head()

,index,법안명,법안내용,tokenized
16487,16487,사립학교법 일부개정법률안,현행법에서는 사립학교의 경영에 관한 중요사항에 대해 이사회의 심의？의결을 거치도록...,현행법 에서 는 사립 학교 의 경영 에 관한 중요 사항 에 대해 이 사회 의 심의 ...
7370,7370,석탄산업법 일부개정법률안,「저탄소 녹색성장 기본법」에 따른 에너지기본계획은 에너지 부문의 모든 분야를 총망...,「 저탄소 녹색 성장 기본법 」 에 따른 에너지 기본 계획 은 에너지 부문 의 모든...
19283,19283,산업표준화법 일부개정법률안,"현행 「산업표준화법 시행규칙」 제16조제2항은 제품인증, 서비스인증(KS)을 받은...","현행 「 산업 표준 화법 시행 규칙 」 제 16 조제 2 항 은 제품 인증 , 서비..."
15385,15385,국토의 계획 및 이용에 관한 법률 일부개정법률안,"통계청에 따르면 저출산·고령화, 1인 가구의 증가로 인하여 2030년 이후부터 우...","통계청 에 따르 면 저출산 · 고령화 , 1 인 가구 의 증가 로 인하 여 2030..."
18459,18459,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,최근 백제역사지구의 세계유산 등재 등 한국의 세계유산 등재 확대와 체계적인 보존·...,최근 백제 역사 지구 의 세계유산 등재 등 한국 의 세계 유산 등 재 확대 와 체계...


In [38]:
토끼원본 = ''
for i in df_토끼['법안내용'].values:
    토끼원본 = 토끼원본 + "{} \n \n".format(i)
    
토끼tokenized = ''
for i in df_토끼['tokenized'].values:
    토끼tokenized = 토끼tokenized + "{} \n \n".format(i)
    
################################################################################

참새원본 = ''
for i in df_참새['법안내용'].values:
    참새원본 = 참새원본 + "{} \n \n".format(i)
    
참새tokenized = ''
for i in df_참새['tokenized'].values:
    참새tokenized = 참새tokenized + "{} \n \n".format(i)
    
################################################################################

병아리원본 = ''
for i in df_병아리['법안내용'].values:
    병아리원본 = 병아리원본 + "{} \n \n".format(i)
    
병아리tokenized = ''
for i in df_병아리['tokenized'].values:
    병아리tokenized = 병아리tokenized + "{} \n \n".format(i)
    
################################################################################

거북이원본 = ''
for i in df_거북이['법안내용'].values:
    거북이원본 = 거북이원본 + "{} \n \n".format(i)
    
거북이tokenized = ''
for i in df_거북이['tokenized'].values:
    거북이tokenized = 거북이tokenized + "{} \n \n".format(i)

## 텍스트 파일로 송출~

In [39]:
f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/토끼(원본).txt", 'w')
f.write(토끼원본)
f.close()

f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/토끼(tokenized).txt", 'w')
f.write(토끼tokenized)
f.close()

################################################################################

f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/참새(원본).txt", 'w')
f.write(참새원본)
f.close()

f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/참새(tokenized).txt", 'w')
f.write(참새tokenized)
f.close()

################################################################################

f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/병아리(원본).txt", 'w')
f.write(병아리원본)
f.close()

f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/병아리(tokenized).txt", 'w')
f.write(병아리tokenized)
f.close()

################################################################################

f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/거북이(원본).txt", 'w')
f.write(거북이원본)
f.close()

f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/거북이(tokenized).txt", 'w')
f.write(거북이tokenized)
f.close()